# Import e config

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc 

import pandas as pd
from io import StringIO

In [ ]:
driver = uc.Chrome()
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)

# Vereadores

In [5]:
driver.get('https://camaragaruva.atende.net/transparencia/item/embed/data/eyJjb2RpZ28iOiIxOCIsInRpcG8iOiIxIiwiZ3J1cG8iOiI0In0=/item/agente-publico')

## Funcoes

Coloca manualmente no Filtro: AGENTE POLITICO e deixe para cargo

In [15]:
def selecionar_mes_ano_index(i):
    locator = 'select[aria-label="Mês/Ano"]'

    texto = driver.find_element(By.CSS_SELECTOR, locator).get_property('children')
    select = Select(driver.find_element(By.CSS_SELECTOR, locator))
    select.select_by_index(i)
    return texto[i].text

In [17]:
selecionar_mes_ano_index(0)

'10/2025'

In [80]:
def df_contracheque():
    sio = StringIO(driver.page_source)
    df = pd.read_html(sio, attrs={'class': 'dados_consulta dados subcontexto_acessibilidade'}, decimal=',', thousands='.')[0]
    df = df.iloc[:, 0:12]
    colunas = ['funcionario', 'contrato', 'nome_funcionario', 'cargo', 'centro_custo', 'afastamento', 'recisao', 'ferias', 'decimo_terceiro', 'salario_bruto', 'descontos', 'salario_liquido' ]
    df.columns = colunas
    return df

In [84]:
def consultar():
    locator_consultar = '#janela_1763744990368 > article > div.form_consulta.consulta-padrao.full.estrutura_consulta_ignora_conteudo.auto_adaptable_flex > aside.div_area_filtros_consulta.area-filtros > div > div:nth-child(3) > table > tbody > tr > td.area_botao_consulta > div > span > span.label_botao_acao.botao-label'
    driver.find_element(By.CSS_SELECTOR, locator_consultar).click()

In [ ]:
from random import uniform
import time

def time_random(a,b):
    '''
    A = número inicial
    B = Número final
    Retorna numero quebrado entre esses números
    '''
    numero = round(uniform(a,b),2)
    print(f'Pausa de {numero} segundos!')
    espera = time.sleep(numero)
    return espera

## Scraping

In [ ]:
referencia = ''
index = 0
locator_table = 'table.dados_consulta.dados.subcontexto_acessibilidade'
tabela = driver.find_element(By.CSS_SELECTOR, locator)

dfs = []
while referencia != '01/2020':
    tabela = driver.find_element(By.CSS_SELECTOR, locator_table)
    print('\n')
    referencia = selecionar_mes_ano_index(index)
    print(f'Selecionando ano {referencia}')
    print('Consultando..')
    consultar()
    
    print('Esperando a tabela mudar..')
    espera_sair_tabela = wait.until(EC.staleness_of(tabela))
    tabela = driver.find_element(By.CSS_SELECTOR, locator)
    time_random(2,4)
    
    df = df_contracheque()
    df['referencia'] = referencia
    dfs.append(df)
    print('Dados coletados!')
    print(df.head(1))
    
    index += 1

df_final = pd.concat(dfs)

In [87]:
import os 
nome_arquivo = 'vereadores_garuva.csv'
caminho = os.path.join('data',nome_arquivo )
df_final.to_csv(caminho, index=False, decimal=',')

# Prefeito

In [89]:
def df_contracheque_prefeito():
    sio = StringIO(driver.page_source)
    df = pd.read_html(sio, attrs={'class': 'dados_consulta dados subcontexto_acessibilidade'}, decimal=',', thousands='.')[0]
    df = df.iloc[:, 0:11]
    colunas = ['funcionario', 'contrato', 'nome_funcionario', 'cargo', 'centro_custo', 'afastamento', 'recisao', 'ferias', 'salario_bruto', 'descontos', 'salario_liquido' ]
    df.columns = colunas
    return df

In [93]:
def consultar_prefeito():
    locator_consultar = '#janela_1763746077697 > article > div.form_consulta.consulta-padrao.full.estrutura_consulta_ignora_conteudo.auto_adaptable_flex > aside.div_area_filtros_consulta.area-filtros > div > div:nth-child(3) > table > tbody > tr > td.area_botao_consulta > div > span > span.label_botao_acao.botao-label'
    driver.find_element(By.CSS_SELECTOR, locator_consultar).click()

In [96]:
referencia = ''
index = 0
locator_table = 'table.dados_consulta.dados.subcontexto_acessibilidade'
tabela = driver.find_element(By.CSS_SELECTOR, locator)

dfs = []
while referencia != '01/2020':
    tabela = driver.find_element(By.CSS_SELECTOR, locator_table)
    print('\n')
    referencia = selecionar_mes_ano_index(index)
    print(f'Selecionando ano {referencia}')
    print('Consultando..')
    consultar_prefeito()
    
    print('Esperando a tabela mudar..')
    espera_sair_tabela = wait.until(EC.staleness_of(tabela))
    tabela = driver.find_element(By.CSS_SELECTOR, locator)
    time_random(2,4)
    
    df = df_contracheque_prefeito()
    df['referencia'] = referencia
    dfs.append(df)
    print('Dados coletados!')
    print(df.head(1))
    
    index += 1

df_final = pd.concat(dfs)



Selecionando ano 10/2025
Consultando..
Esperando a tabela mudar..
Pausa de 3.45 segundos!
Dados coletados!
   funcionario  contrato       nome_funcionario     cargo  \
0        10626         0  PLOTINO DE BITENCOURT  PREFEITO   

               centro_custo afastamento  recisao  ferias  salario_bruto  \
0  Departamento de Gabinete         Não      0.0     0.0        30390.0   

   descontos  salario_liquido referencia  
0    8138.44         22251.56    10/2025  


Selecionando ano 09/2025
Consultando..
Esperando a tabela mudar..
Pausa de 2.36 segundos!
Dados coletados!
   funcionario  contrato       nome_funcionario     cargo  \
0        10626         0  PLOTINO DE BITENCOURT  PREFEITO   

               centro_custo afastamento  recisao  ferias  salario_bruto  \
0  Departamento de Gabinete         Não      0.0     0.0        30390.0   

   descontos  salario_liquido referencia  
0    8138.44         22251.56    09/2025  


Selecionando ano 08/2025
Consultando..
Esperando a tabela mu

In [97]:
import os 
nome_arquivo = 'prefeito_garuva.csv'
caminho = os.path.join('data',nome_arquivo )
df_final.to_csv(caminho, index=False, decimal=',')